# Graphical Model Tutorial - Named Entity Recognition

Ruijie Wang, Pascal Severin Andermatt | 12.10.2022  
Based on [Named Entity Recognition and Classification with Scikit-Learn](https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2), [ACNER](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus), [Hidden Markov Model for POS-tagging](https://medium.com/mlearning-ai/introduction-to-hidden-markov-model-hmm-with-simple-ner-d1353ff35842) and [sklearn-crfsuite](https://sklearn-crfsuite.readthedocs.io/en/latest/index.html).

## 1. Named Entity Recognition

* **Named Entity Recognition (NER)** is the process of recognizing information units like **names**, including person, organization and location names, and **numeric expressions** including time, date, money and percent expressions from **unstructured text**.

    * Who is the director of **Batman 1989**?


* NER is usually converted to the problem of **tagging** each word in the given text. 

    * For example, the tagging of the question **Who is the director of Batman 1989?** could be:
        
        | Id | Word | Tag | Description |
        | ---- | ---- | ---- | ---- |
        | 0 | Who | O | others |
        | 1 | is | O | - |
        | 2 | the | O | - |
        | 3 | director | O | - |
        | 4 | of | O | - |
        | 5 | Batman | **B-mov** | mov = movie |
        | 6 | 1989 | **I-mov** | - |
        | 7 | ? | O | - |

    * The above tages follow the [**Inside–outside–beginning (IOB) format**](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)#cite_note-2): 
    
        * An O tag indicates that a token belongs to **no chunk**. In NER, it means the word is not an entity.
        
        * The B- prefix before a tag indicates that the tag is the **beginning** of a chunk.
        
        * The I- prefix before a tag indicates that the tag is **inside** a chunk.

## 2. Packages

* We will use **[pandas](https://pandas.pydata.org/)** and **[numpy](https://numpy.org/)** to read and manipulate the data.

In [10]:
!pip install pandas
!pip install numpy


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


* **[Scikit-learn](https://scikit-learn.org/stable/)** and **[sklearn-crfsuite](https://sklearn-crfsuite.readthedocs.io/en/latest/)** will be used to train baseline, HMM and CRF models.

In [11]:
!pip install -U 'scikit-learn<0.24'
!pip install sklearn-crfsuite

The system cannot find the file specified.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


- import required packages and modules

In [12]:
import pandas as pd
import numpy as np
import sklearn_crfsuite

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

from collections import Counter

## 3. Dataset

* **[Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)**

* An annotated corpus for Named Entity Recognition based on [GMB (Groningen Meaning Bank)](https://gmb.let.rug.nl/).

* There are mainly 8 types of tags:  

    | Tag | Description | Examples |
    | ---- | ---- | ---- |
    | geo | Geographical Entity | Paris, Washington DC |
    | org | Organization | APEC, WTO, United Nations |
    | per | Person | President George Bush |
    | gpe | Geopolitical Entity | U.S., France, Japan |
    | tim | Time indicator | July, Wednesday |
    | art | Artifact | Internet, International Space Station |
    | eve | Event | World War I, Tennis Masters Cup |
    | nat | Natural Phenomenon | Hurricane Katrina |

* There are two .csv files in the corpus:

    * **ner_dataset.csv** includes two features: `word` and `POS`. `POS` refers to part-of-speech tags. You can find a set of tags and descriptions at https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

    * **ner.csv** provides more features such as `shape`, `prev-word`, `next-word` that you can use to achieve better performance.

- Let's first use pandas to read the csv file. 

In [13]:
#graph = rdflib.Graph().parse('C:/Users/Memir/OneDrive/Masaüstü/ATAI/Codes/DataSet/ddis-movie-graph.nt/14_graph.nt', format='turtle')


In [14]:
df = pd.read_csv('ner_dataset.csv', encoding = "ISO-8859-1")
df = df[:10000]  # only take the first 10,000 records for this tutorial
df = df.fillna(method='ffill')
df[:100]

C:\Users\Memir\AppData\Local\Temp\ipykernel_4836\1828568199.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
95,Sentence: 5,'s,POS,O
96,Sentence: 5,ruling,VBG,O
97,Sentence: 5,Labor,NNP,B-org
98,Sentence: 5,Party,NNP,I-org


In [15]:
print("- number of sentences: {},\n- number of unique words: {},\n- number of unique POS tags: {},\n- number of unique NER tags: {}".format(df['Sentence #'].nunique(), df.Word.nunique(), df.POS.nunique(), df.Tag.nunique()))

- number of sentences: 457,
- number of unique words: 2746,
- number of unique POS tags: 39,
- number of unique NER tags: 17


In [16]:
# show the distribution of NER tags
df.groupby('Tag').size().reset_index(name='Counts')

,Tag,Counts
0,B-art,28
1,B-eve,10
2,B-geo,244
3,B-gpe,303
4,B-nat,5
5,B-org,176
6,B-per,160
7,B-tim,149
8,I-art,20
9,I-eve,10


## 4. A Linear classifier with SGD training

- A **support vector machine (SVM)** learns hyperplanes to seperate words with different tags.

- Each word is classified independently by the classifier based on its features.

- Dependencies between words are not considered in this tutorial.

In [17]:
# only leave Word and POS as input features
X = df.drop('Tag', axis=1).drop('Sentence #', axis=1)
X

,Word,POS
0,Thousands,NNS
1,of,IN
2,demonstrators,NNS
3,have,VBP
4,marched,VBN
...,...,...
9995,war,NN
9996,crimes,NNS
9997,tribunal,NN
9998,in,IN


In [18]:
# vectorize X
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
print("--- the shape of X after vectorization: {}".format(X.shape))
X

--- the shape of X after vectorization: (10000, 2785)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
# NER targets
y = df.Tag.values
print("--- the shape of y: {}".format(X.shape))
y[:22]

--- the shape of y: (10000, 2785)


array(['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O',
       'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O'],
      dtype=object)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)

print("--- number of training and test words: {}, {}".format( X_train.shape[0], X_test.shape[0]))
print("--- dimension of input features: {}".format(X_train.shape[1]))

--- number of training and test words: 6700, 3300
--- dimension of input features: 2785


In [21]:
# leave all other parameters to default
sgd = SGDClassifier(loss="hinge", max_iter=1000)  # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

# train the model
sgd.fit(X_train, y_train)

classifier_res = classification_report(y_pred=sgd.predict(X_test), y_true=y_test)
print(classifier_res)

              precision    recall  f1-score   support

       B-art       1.00      0.22      0.36         9
       B-eve       0.50      0.33      0.40         3
       B-geo       0.28      0.71      0.40        69
       B-gpe       0.76      0.67      0.71       102
       B-nat       0.00      0.00      0.00         0
       B-org       0.82      0.51      0.63        63
       B-per       0.83      0.61      0.70        41
       B-tim       0.94      0.92      0.93        52
       I-art       0.00      0.00      0.00        10
       I-eve       1.00      0.33      0.50         3
       I-geo       0.75      0.27      0.40        11
       I-gpe       0.67      0.33      0.44         6
       I-nat       0.00      0.00      0.00         1
       I-org       0.72      0.45      0.55        47
       I-per       0.86      0.38      0.53        66
       I-tim       0.00      0.00      0.00         4
           O       0.99      1.00      0.99      2813

    accuracy              

c:\Users\Memir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Memir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Memir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

* F1-score is computed based on precision and recall: $2*((precision*recall)/(precision+recall))$
* Macro Average Precision is the average precision w.r.t all classes.
* Weighted Average Precision is the weighted average precision of all classes.

## HMM-based POS tagging

* Tags and words are respectively considered as hidden states and observations.

* Two matrices would be needed for HMM modeling:

    * transition probability matrix
    
    * observation probability matrix

In [22]:
def collate(dataframe):
    agg_func = lambda s: [(w, t) for w, t in zip(s['Word'].values.tolist(), s['Tag'].values.tolist())]
    grouped = dataframe.groupby('Sentence #').apply(agg_func)
    return list(grouped)

print("--- original dataframe:\n")
print(df)
sentences = collate(df)
print("\n--- number of sentences: {}\n".format(len(sentences)))
sentences[0:2]

--- original dataframe:

         Sentence #           Word  POS Tag
0       Sentence: 1      Thousands  NNS   O
1       Sentence: 1             of   IN   O
2       Sentence: 1  demonstrators  NNS   O
3       Sentence: 1           have  VBP   O
4       Sentence: 1        marched  VBN   O
...             ...            ...  ...  ..
9995  Sentence: 457            war   NN   O
9996  Sentence: 457         crimes  NNS   O
9997  Sentence: 457       tribunal   NN   O
9998  Sentence: 457             in   IN   O
9999  Sentence: 457            The   DT   O

[10000 rows x 4 columns]

--- number of sentences: 457



C:\Users\Memir\AppData\Local\Temp\ipykernel_4836\1555979887.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = dataframe.groupby('Sentence #').apply(agg_func)


[[('Thousands', 'O'),
  ('of', 'O'),
  ('demonstrators', 'O'),
  ('have', 'O'),
  ('marched', 'O'),
  ('through', 'O'),
  ('London', 'B-geo'),
  ('to', 'O'),
  ('protest', 'O'),
  ('the', 'O'),
  ('war', 'O'),
  ('in', 'O'),
  ('Iraq', 'B-geo'),
  ('and', 'O'),
  ('demand', 'O'),
  ('the', 'O'),
  ('withdrawal', 'O'),
  ('of', 'O'),
  ('British', 'B-gpe'),
  ('troops', 'O'),
  ('from', 'O'),
  ('that', 'O'),
  ('country', 'O'),
  ('.', 'O')],
 [('Iranian', 'B-gpe'),
  ('officials', 'O'),
  ('say', 'O'),
  ('they', 'O'),
  ('expect', 'O'),
  ('to', 'O'),
  ('get', 'O'),
  ('access', 'O'),
  ('to', 'O'),
  ('sealed', 'O'),
  ('sensitive', 'O'),
  ('parts', 'O'),
  ('of', 'O'),
  ('the', 'O'),
  ('plant', 'O'),
  ('Wednesday', 'B-tim'),
  (',', 'O'),
  ('after', 'O'),
  ('an', 'O'),
  ('IAEA', 'B-org'),
  ('surveillance', 'O'),
  ('system', 'O'),
  ('begins', 'O'),
  ('functioning', 'O'),
  ('.', 'O')]]

In [23]:
train_sentences, test_sentences = train_test_split(sentences, test_size=0.3, random_state=0)
print('- number of training sentences: ', len(train_sentences))
print('- number of testing sentences: ', len(test_sentences))

- number of training sentences:  319
- number of testing sentences:  138


In [24]:
# count of word-tag occurrences
token_list = [word for sentence in train_sentences for word in sentence]
tokens, tags = list(zip(*token_list))
print("--- number of training tokens: {}".format(len(tokens)))


hmm_df = pd.DataFrame(tokens, tags).reset_index().rename(columns={0: 'tokens', 'index': 'tag'})
hmm_df['values'] = 1
hmm_df

--- number of training tokens: 6943


,tag,tokens,values
0,B-per,Mr.,1
1,I-per,Sivaram,1
2,O,",",1
3,O,who,1
4,O,was,1
...,...,...,...
6938,O,contact,1
6939,O,with,1
6940,O,dead,1
6941,O,birds,1


In [25]:
# compute the observation probability matrix
hmm_df = hmm_df.pivot_table(index='tag', columns='tokens', aggfunc='sum').fillna(0)

sum_ = hmm_df.sum(axis=1)
hmm_df = hmm_df.div(sum_, axis=0)
hmm_df.columns = hmm_df.columns.droplevel(0)

hmm_df

hmm_df.style.highlight_max(color = 'lightgreen', axis = 1)

In [26]:
# compute transition probability matrix
df2 = []
for sentence in train_sentences:
    for index in range(len(sentence) - 1):
        first_token = sentence[index][1]
        next_token = sentence[index + 1][1]
        df2.append({'first_token': first_token, 'next_token': next_token})

df2 = pd.DataFrame(df2)

df2['values'] = 1
df2 = df2.pivot_table(index='first_token', columns='next_token', aggfunc='sum').fillna(0)
df2 = df2.div(df2.sum(axis=1), axis=0)
df2.columns = df2.columns.droplevel(0)
df_colums = df2.columns.tolist()

# compute probability distribution of the first token
start_tag = '<s>'
prob_start = {'B': 0, 'I': 0, 'O': 0}
for sentence in train_sentences:
    # summarize IOB tags to only keep the first letter (eg. B-art -> B)
    tag = sentence[0][1]
    prob_start[tag[0]] += 1
prob_start = {k: v / sum(prob_start.values()) for k, v in prob_start.items()}
print("Probabilities that the first token is a B, I, or O: {}".format(prob_start))
probs_data = [prob_start[i[0]] for i in df_colums]

df2 = pd.concat([pd.DataFrame(columns=df_colums, index=[start_tag], data=[probs_data]), df2], axis=0)

df2.style.highlight_max(color = 'lightgreen', axis = 1)

Probabilities that the first token is a B, I, or O: {'B': 0.29780564263322884, 'I': 0.0, 'O': 0.7021943573667712}


,B-art,B-eve,B-geo,B-gpe,B-nat,B-org,B-per,B-tim,I-art,I-eve,I-geo,I-gpe,I-org,I-per,I-tim,O
,0.297806,0.297806,0.297806,0.297806,0.297806,0.297806,0.297806,0.297806,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.702194
B-art,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.391304,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.608696
B-eve,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.875000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000
B-geo,0.000000,0.000000,0.000000,0.005882,0.000000,0.000000,0.000000,0.035294,0.000000,0.000000,0.076471,0.000000,0.000000,0.000000,0.000000,0.882353
B-gpe,0.000000,0.000000,0.000000,0.000000,0.000000,0.026549,0.088496,0.000000,0.000000,0.000000,0.000000,0.061947,0.000000,0.000000,0.000000,0.823009
B-nat,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
B-org,0.007874,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.440945,0.000000,0.000000,0.551181
B-per,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.801887,0.000000,0.198113
B-tim,0.000000,0.000000,0.009091,0.009091,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.081818,0.900000
I-art,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.437500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.562500


In [27]:
def predict(sentence):
    tokens = [token for (token, _) in sentence]
    tags = [] # empty because we will fill it with the predicted tags
    previous_tag = start_tag

    for token in tokens:
        if token not in hmm_df.columns:
            previous_tag = 'O'
        else:
            matrix = hmm_df[token].multiply(df2.loc[previous_tag])
            previous_tag = hmm_df.index[np.argmax(matrix)]
        token_tag_pair = (token, previous_tag)
        tags.append(token_tag_pair)
    
    return tags

sentence = test_sentences[1]
print(sentence)
print(f"original sentence: {sentence}\n")
print(f"predicted tags: {predict(sentence)}")


[('Kony', 'B-per'), ('is', 'O'), ('one', 'O'), ('of', 'O'), ('five', 'O'), ('LRA', 'B-org'), ('officials', 'O'), ('sought', 'O'), ('by', 'O'), ('the', 'O'), ('International', 'B-org'), ('Criminal', 'I-org'), ('Court', 'I-org'), ('for', 'O'), ('alleged', 'O'), ('war', 'O'), ('crimes', 'O'), ('.', 'O')]
original sentence: [('Kony', 'B-per'), ('is', 'O'), ('one', 'O'), ('of', 'O'), ('five', 'O'), ('LRA', 'B-org'), ('officials', 'O'), ('sought', 'O'), ('by', 'O'), ('the', 'O'), ('International', 'B-org'), ('Criminal', 'I-org'), ('Court', 'I-org'), ('for', 'O'), ('alleged', 'O'), ('war', 'O'), ('crimes', 'O'), ('.', 'O')]

predicted tags: [('Kony', 'B-art'), ('is', 'O'), ('one', 'O'), ('of', 'O'), ('five', 'O'), ('LRA', 'B-org'), ('officials', 'O'), ('sought', 'O'), ('by', 'O'), ('the', 'O'), ('International', 'B-org'), ('Criminal', 'O'), ('Court', 'B-art'), ('for', 'O'), ('alleged', 'O'), ('war', 'O'), ('crimes', 'O'), ('.', 'O')]


In [28]:
import matplotlib.pyplot as plt
import seaborn as sn
def evaluate(test_sentences):
    y_true = []
    y_pred = []
    for sentence in test_sentences:
        y_true.extend([tag for (_, tag) in sentence])
        y_pred.extend([tag for (_, tag) in predict(sentence)])

    return classification_report(y_true, y_pred)

hmm_res = evaluate(test_sentences)
print("--- performance of the HMM model")
print(hmm_res)

ModuleNotFoundError: No module named 'seaborn'

## Conditional Random Fields (CRFs)

* The above HMM model only considers transition between hidden states (tags) and transition between hidden states (tags) and words.

* There are actually many other features that we can use, e.g., POS tags of words and dependences between words.

* How can we improve the NER performance by considering more features?

In [29]:
def collate(dataframe):
    agg_func = lambda s: [(w, pos, t) for w, pos, t in zip(s['Word'].values.tolist(), s['POS'].values.tolist(), s['Tag'].values.tolist())]
    grouped = dataframe.groupby('Sentence #').apply(agg_func)
    return list(grouped)

print("--- original dataframe:\n")
print(df)
sentences = collate(df)
print("\n--- number of sentences: {}\n".format(len(sentences)))
sentences[0:2]

--- original dataframe:

         Sentence #           Word  POS Tag
0       Sentence: 1      Thousands  NNS   O
1       Sentence: 1             of   IN   O
2       Sentence: 1  demonstrators  NNS   O
3       Sentence: 1           have  VBP   O
4       Sentence: 1        marched  VBN   O
...             ...            ...  ...  ..
9995  Sentence: 457            war   NN   O
9996  Sentence: 457         crimes  NNS   O
9997  Sentence: 457       tribunal   NN   O
9998  Sentence: 457             in   IN   O
9999  Sentence: 457            The   DT   O

[10000 rows x 4 columns]

--- number of sentences: 457



C:\Users\Memir\AppData\Local\Temp\ipykernel_4836\2027667274.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = dataframe.groupby('Sentence #').apply(agg_func)


[[('Thousands', 'NNS', 'O'),
  ('of', 'IN', 'O'),
  ('demonstrators', 'NNS', 'O'),
  ('have', 'VBP', 'O'),
  ('marched', 'VBN', 'O'),
  ('through', 'IN', 'O'),
  ('London', 'NNP', 'B-geo'),
  ('to', 'TO', 'O'),
  ('protest', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('war', 'NN', 'O'),
  ('in', 'IN', 'O'),
  ('Iraq', 'NNP', 'B-geo'),
  ('and', 'CC', 'O'),
  ('demand', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('withdrawal', 'NN', 'O'),
  ('of', 'IN', 'O'),
  ('British', 'JJ', 'B-gpe'),
  ('troops', 'NNS', 'O'),
  ('from', 'IN', 'O'),
  ('that', 'DT', 'O'),
  ('country', 'NN', 'O'),
  ('.', '.', 'O')],
 [('Iranian', 'JJ', 'B-gpe'),
  ('officials', 'NNS', 'O'),
  ('say', 'VBP', 'O'),
  ('they', 'PRP', 'O'),
  ('expect', 'VBP', 'O'),
  ('to', 'TO', 'O'),
  ('get', 'VB', 'O'),
  ('access', 'NN', 'O'),
  ('to', 'TO', 'O'),
  ('sealed', 'JJ', 'O'),
  ('sensitive', 'JJ', 'O'),
  ('parts', 'NNS', 'O'),
  ('of', 'IN', 'O'),
  ('the', 'DT', 'O'),
  ('plant', 'NN', 'O'),
  ('Wednesday', 'NNP', 'B-tim'),
  ('

In [30]:
train_sentences, test_sentences = train_test_split(sentences, test_size=0.3, random_state=0)
print('- number of training sentences: ', len(train_sentences))
print('- number of testing sentences: ', len(test_sentences))
print(train_sentences[0])

- number of training sentences:  319
- number of testing sentences:  138
[('Mr.', 'NNP', 'B-per'), ('Sivaram', 'NNP', 'I-per'), (',', ',', 'O'), ('who', 'WP', 'O'), ('was', 'VBD', 'O'), ('also', 'RB', 'O'), ('brutally', 'RB', 'O'), ('attacked', 'VBN', 'O'), ('in', 'IN', 'O'), ('2001', 'CD', 'B-tim'), (',', ',', 'O'), ('was', 'VBD', 'O'), ('found', 'VBN', 'O'), ('near', 'IN', 'O'), ('a', 'DT', 'O'), ('lake', 'NN', 'O'), ('gagged', 'VBN', 'O'), ('with', 'IN', 'O'), ('gunshot', 'NN', 'O'), ('wounds', 'NNS', 'O'), ('to', 'TO', 'O'), ('the', 'DT', 'O'), ('head', 'NN', 'O'), ('.', '.', 'O')]


* We consider the following features in the CRF:

    * POS tags
    * Word parts
    * Lower/title/upper flags
    * IBO prefixes
    * Features of neighboring words


* As required by sklearn-crfsuite, each sentence should be converted to a list of dicts.

In [31]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'word.lower()': word.lower(),  # the word in lowercase
        'word[-3:]': word[-3:],  # last three characters
        'word[-2:]': word[-2:],  # last two characters
        'word.isupper()': word.isupper(),  # true, if the word is in uppercase
        'word.istitle()': word.istitle(),  # true, if the first character is in uppercase and remaining characters are in lowercase
        'word.isdigit()': word.isdigit(),  # true, if all characters are digits
        'postag': postag,  # POS tag
        'postag[:2]': postag[:2],  # IOB prefix
    }
    
    if i > 0:
        word1 = sent[i-1][0]  # the previous word
        postag1 = sent[i-1][1]  # POS tag of the previous word
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })  # add some features of the previous word
    else:
        features['BOS'] = True  # BOS: begining of the sentence
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]  # the next word
        postag1 = sent[i+1][1]  # POS tag of the next word
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })  # add some features of the next word
    else:
        features['EOS'] = True  # EOS: end of the sentence
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for _, _, label in sent]

In [32]:
# example
print("--- extracted features of the first word of the first training sentence:\n")
for key, value in sent2features(train_sentences[0])[0].items():
    print(key, value)

print("\n--- extracted features of the second word of the first training sentence:\n")
for key, value in sent2features(train_sentences[0])[1].items():
    print(key, value)
    
print("\n--- NER tag of the first word:\n")
print(sent2labels(train_sentences[0])[0])

--- extracted features of the first word of the first training sentence:

word.lower() mr.
word[-3:] Mr.
word[-2:] r.
word.isupper() False
word.istitle() True
word.isdigit() False
postag NNP
postag[:2] NN
BOS True
+1:word.lower() sivaram
+1:word.istitle() True
+1:word.isupper() False
+1:postag NNP
+1:postag[:2] NN

--- extracted features of the second word of the first training sentence:

word.lower() sivaram
word[-3:] ram
word[-2:] am
word.isupper() False
word.istitle() True
word.isdigit() False
postag NNP
postag[:2] NN
-1:word.lower() mr.
-1:word.istitle() True
-1:word.isupper() False
-1:postag NNP
-1:postag[:2] NN
+1:word.lower() ,
+1:word.istitle() False
+1:word.isupper() False
+1:postag ,
+1:postag[:2] ,

--- NER tag of the first word:

B-per


In [34]:
# X and y in the required format
X_train, y_train = [sent2features(s) for s in train_sentences], [sent2labels(s) for s in train_sentences]
X_test, y_test = [sent2features(s) for s in test_sentences], [sent2labels(s) for s in test_sentences]

- Train a CRF model

In [35]:
crf = sklearn_crfsuite.CRF(
    algorithm='l2sgd',  # l2sgd: Stochastic Gradient Descent with L2 regularization term
    max_iterations=1000,  # maximum number of iterations
)

crf.fit(X_train, y_train)

CRF(algorithm='l2sgd', max_iterations=1000)

In [36]:
y_pred = crf.predict(X_test)

print("--- performance of the CRF model")
print(metrics.flat_classification_report(y_test, y_pred))

print("--- performance of the HMM model")
print(hmm_res)

--- performance of the CRF model
              precision    recall  f1-score   support

       B-art       0.67      0.40      0.50         5
       B-eve       0.00      0.00      0.00         2
       B-geo       0.73      0.74      0.73        73
       B-gpe       0.77      0.84      0.81        77
       B-nat       0.00      0.00      0.00         2
       B-org       0.79      0.69      0.74        49
       B-per       0.82      0.91      0.86        54
       B-tim       0.97      0.77      0.86        39
       I-art       0.00      0.00      0.00         4
       I-eve       0.00      0.00      0.00         1
       I-geo       0.71      0.31      0.43        16
       I-gpe       0.75      1.00      0.86         3
       I-nat       0.00      0.00      0.00         2
       I-org       0.78      0.67      0.72        48
       I-per       0.83      1.00      0.91        65
       I-tim       1.00      1.00      1.00         1
           O       0.99      1.00      1.00     

c:\Users\Memir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Memir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Memir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

NameError: name 'hmm_res' is not defined

- The performance has been significantly improved by considering more features of given words and dependencies between neighboring words.

## Discussion - What has the CRF model learned?

- Check the learned dependencies between words:

In [37]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
        
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
O      -> O       4.053447
B-org  -> I-org   3.168619
B-per  -> I-per   3.097295
I-org  -> I-org   2.802774
I-per  -> I-per   2.570287
B-art  -> I-art   2.136077
B-gpe  -> I-gpe   2.133255
B-eve  -> I-eve   1.993109
B-geo  -> I-geo   1.964260
I-art  -> I-art   1.796720
O      -> B-tim   1.602925
O      -> B-gpe   1.593524
O      -> B-geo   1.468808
B-tim  -> I-tim   1.378199
O      -> B-org   1.234328
I-gpe  -> I-gpe   1.087601
O      -> B-art   1.046549
I-tim  -> I-tim   0.925341
B-gpe  -> B-per   0.870624
I-eve  -> I-eve   0.731077

Top unlikely transitions:
B-geo  -> B-gpe   0.224478
B-tim  -> B-geo   0.220808
I-tim  -> B-geo   0.182237
I-org  -> B-tim   0.170328
I-geo  -> B-tim   0.100897
B-org  -> O       0.096087
O      -> B-nat   0.093481
I-org  -> O       0.059435
I-org  -> B-per   0.040815
B-per  -> O       -0.069829
I-tim  -> O       -0.123520
I-gpe  -> O       -0.133471
I-per  -> O       -0.140463
B-nat  -> O       -0.142388
B-art  -> O       -0.15111

- Check how the CRF considers given features of words:

In [38]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%-6s -> %-7s %0.6f" % (attr, label, weight))
        
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
BOS    -> O       3.313032
word[-3:]:day -> B-tim   2.406988
word[-2:]:ay -> B-tim   2.381390
postag:NN -> O       2.057323
word.istitle() -> B-gpe   1.981326
-1:word.lower():in -> B-geo   1.549408
postag:JJ -> B-gpe   1.429421
postag:NNS -> O       1.290839
postag[:2]:JJ -> B-gpe   1.287888
postag[:2]:VB -> O       1.280169
word.isdigit() -> B-tim   1.267905
-1:word.lower():in -> B-tim   1.232545
word.isupper() -> B-org   1.223796
EOS    -> O       1.197062
-1:postag[:2]:NN -> O       1.171598
postag:NNP -> B-geo   1.142281
word.isdigit() -> I-tim   1.127104
word.lower():u.s. -> B-gpe   1.024279
word[-3:]:.S. -> B-gpe   1.024279
word[-2:]:S. -> B-gpe   1.024279
word[-3:]:ban -> B-org   1.006687
postag:CD -> I-tim   1.005174
postag[:2]:CD -> I-tim   1.005174
postag[:2]:NN -> B-org   1.002154
word[-2:]:na -> B-gpe   0.992988
word[-2:]:an -> B-gpe   0.992616
word.istitle() -> B-geo   0.946319
postag:CD -> B-tim   0.922046
postag[:2]:CD -> B-tim   0.922046
postag:DT -> O    

* [ELI5](https://eli5.readthedocs.io/en/latest/)
    * ELI5 is a Python package that can be used to check weights of `sklearn_crfsuite.CRF` models.

In [44]:
!pip install eli5
!pip install spacy
import eli5
!spacy download en_core_web_trf


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


ImportError: cannot import name 'if_delegate_has_method' from 'sklearn.utils.metaestimators' (c:\Users\Memir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\metaestimators.py)

In [39]:
# inspect model weights
eli5.show_weights(crf, top=10)  # top: number of features to show

/Users/wangruijie/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,4.017,1.038,0.0,0.583,0.0,1.442,0.0,1.609,0.0,0.092,1.263,0.0,0.597,0.0,1.604,0.0
B-art,-0.156,0.0,2.135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-art,-0.331,0.0,1.802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-eve,-0.602,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-eve,-0.221,0.0,0.0,0.0,0.735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-geo,0.401,0.0,0.0,0.0,0.0,0.0,1.963,0.229,0.0,0.0,0.0,0.0,0.0,0.0,0.586,0.0
I-geo,-0.306,0.0,0.0,0.0,0.0,0.0,0.685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.098,0.0
B-gpe,0.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.136,0.0,0.472,0.0,0.87,0.0,0.0,0.0
I-gpe,-0.133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.095,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-nat,-0.144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


* We can also specify a subset of tags to check.

In [40]:
eli5.show_weights(crf, top=10, targets=['O', 'B-org', 'I-per'])

- Or check only some of the features for all tags.

In [41]:
eli5.show_weights(crf, top=10, feature_re='^word\.is',
                  horizontal_layout=False, show=['targets'])

Weight?,Feature
-0.523,word.isupper()
-1.445,word.isdigit()
-2.773,word.istitle()
Weight?,Feature
+0.129,word.istitle()
-0.108,word.isdigit()
-0.329,word.isupper()
Weight?,Feature
+0.111,word.isupper()
+0.099,word.istitle()


## NER for Question Answering

- NER could be very useful for your final project because recognizing named entities in given natural language questions is usually the very first step. 

In [1]:
import rdflib
from rdflib import Graph, URIRef, Literal
from rdflib import Namespace

g = Graph()
n = Namespace("http://example.org/")

tim = n.Tim_Burton
tim_name = Literal("Timothy Walter Burton")
batman = n.Batman_1989


g.add((tim, n.director, batman)) 
g.add((tim, n.label, tim_name))


print("number of triples: {}\n".format(len(g)))
for s, p, o in g:
    print("s: {},\t p: {},\t o: {}\n".format(s, p, o))

number of triples: 2

s: http://example.org/Tim_Burton,	 p: http://example.org/director,	 o: http://example.org/Batman_1989

s: http://example.org/Tim_Burton,	 p: http://example.org/label,	 o: Timothy Walter Burton



- The user poses a question **"who is the director of Batman 1989"**.

In [2]:
question = "who is the director of Batman 1989"


- **"Batman 1989"** can be recognized as an entity via NER.

In [7]:
import spacy
nlp = spacy.load('C:/Users/Memir/OneDrive/Masaüstü/ATAI/Codes/Speakeasybot/.venv/Lib/site-packages/en_core_web_trf')

# The question we're processing
question = "Who is the director of Batman 1989?"

# Process the question using SpaCy's NER pipeline
doc = nlp(question)

# Extract entities
entities = []
for ent in doc.ents:
    # We're interested in entities labeled as "WORK_OF_ART", which is often used for movie titles
    if ent.label_ == "WORK_OF_ART":
        entities.append(ent.text)

# Print the extracted entities
if entities:
    print(f"Entity found: {entities[0]}")
else:
    print("No valid entity found.")


OSError: [E053] Could not read config file from C:\Users\Memir\OneDrive\Masaüstü\ATAI\Codes\Speakeasybot\.venv\Lib\site-packages\en_core_web_trf\config.cfg

In [29]:
'''
-------

implement your NER method

-------
'''

entity = 'Batman 1989' 

- By defining a **question pattern**, the **relation associated with the entity** can be recognized;

In [30]:
import re

# a naive way for matching entities and relations

question_pattern = "who is the (.*) of ENTITY"

print("question pattern: {}\n".format(question_pattern))

question = re.sub(entity, "ENTITY", question.rstrip("?"))  # preprocess the question

relation = re.match(question_pattern, question).group(1)  # match the relation using a pattern

print("recognized relation: {}\n".format(relation))

question pattern: who is the (.*) of ENTITY

recognized relation: director



- Given "Batman 1989" and "director", find the corresponding entity and predicate in the knowledge graph: `<Batman_1989>` and `<director>`

In [31]:
nodes = {}
predicates = {}

for node in g.all_nodes():
    if isinstance(node, URIRef):
        if g.value(node, n.label):
            nodes[node.toPython()] = g.value(node, n.label).toPython()
        else:
            nodes[node.toPython()] = re.sub("http://example.org/", "", node.toPython())

for s, p, o in g:
    predicates[p.toPython()] = re.sub("http://example.org/", "", p.toPython())

print("labeled nodes: {}\n".format(nodes))
print("predicates: {}\n".format(predicates))

labeled nodes: {'http://example.org/Tim_Burton': 'Timothy Walter Burton', 'http://example.org/Batman_1989': 'Batman_1989'}

predicates: {'http://example.org/label': 'label', 'http://example.org/director': 'director'}



In [32]:
!pip install editdistance

   ---------------------------------------- 0.0/79.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/79.9 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/79.9 kB ? eta -:--:--
   ---------- ----------------------------- 20.5/79.9 kB 217.9 kB/s eta 0:00:01
   ------------------------- -------------- 51.2/79.9 kB 327.7 kB/s eta 0:00:01
   ---------------------------------------- 79.9/79.9 kB 446.2 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
import editdistance

tmp = 9999
match_node = ""
print("--- entity matching for \"{}\"\n".format(entity))
for key, value in nodes.items():
    print("edit distance between {} and {}: {}".format(value, entity, editdistance.eval(value, entity)))
    if editdistance.eval(value, entity) < tmp:
        tmp = editdistance.eval(value, entity)
        match_node = key

tmp = 9999
match_pred = ""
print("\n--- relation matching for \"{}\"\n".format(relation))
for key, value in predicates.items():
    print("edit distance between {} and {}: {}".format(value, relation, editdistance.eval(value, relation)))
    if editdistance.eval(value, relation) < tmp:
        tmp = editdistance.eval(value, relation)
        match_pred = key

print("\n--- the matching node of \"{}\" is {}\n".format(entity, match_node))
print("--- the matching predicates of \"{}\" is {}\n".format(relation, match_pred))

--- entity matching for "Batman 1989"

edit distance between Timothy Walter Burton and Batman 1989: 18
edit distance between Batman_1989 and Batman 1989: 1

--- relation matching for "director"

edit distance between label and director: 7
edit distance between director and director: 0

--- the matching node of "Batman 1989" is http://example.org/Batman_1989

--- the matching predicates of "director" is http://example.org/director



- The corresponding SPARQL query can be constructed according to a SPARQL template: `SELECT ?x WHERE {?x, PREDICATE, NODE.}`

In [34]:
query_template = "SELECT DISTINCT ?x ?y WHERE {{ ?x <{}> <{}>. ?x <{}> ?y. }}".format(match_pred, match_node, n.label)

print("--- sparql query: {}".format(query_template))

qres = g.query(query_template)

print("\n--- querying results: ")
for row in qres:
    print(row.x, row.y)
    answer = row.y

--- sparql query: SELECT DISTINCT ?x ?y WHERE { ?x <http://example.org/director> <http://example.org/Batman_1989>. ?x <http://example.org/label> ?y. }

--- querying results: 
http://example.org/Tim_Burton Timothy Walter Burton


- Finally, we can define response templates to generate natural language responses:

In [35]:
answer_template = "Hi, the {} of {} is {}".format(relation, entity, answer)

print("\n--- generated response: {}".format(answer_template))


--- generated response: Hi, the director of Batman 1989 is Timothy Walter Burton
